In [1]:
from sklearn.datasets import load_iris
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [2]:
m = nn.Linear(20,30)

In [3]:
input = torch.ones(128, 20)

In [4]:
output = m(input)

In [5]:
output

tensor([[-0.6515, -1.2852,  0.0750,  ...,  0.4384, -1.2831, -0.4053],
        [-0.6515, -1.2852,  0.0750,  ...,  0.4384, -1.2831, -0.4053],
        [-0.6515, -1.2852,  0.0750,  ...,  0.4384, -1.2831, -0.4053],
        ...,
        [-0.6515, -1.2852,  0.0750,  ...,  0.4384, -1.2831, -0.4053],
        [-0.6515, -1.2852,  0.0750,  ...,  0.4384, -1.2831, -0.4053],
        [-0.6515, -1.2852,  0.0750,  ...,  0.4384, -1.2831, -0.4053]],
       grad_fn=<AddmmBackward>)

In [6]:
a = np.arange(10).astype(np.float32)
a_tensor = torch.from_numpy(a).clone()
m = F.log_softmax(a_tensor)
m



<ipython-input-6-1aed2bd4333a>:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  m = F.log_softmax(a_tensor)


tensor([-9.4586, -8.4586, -7.4586, -6.4586, -5.4586, -4.4586, -3.4586, -2.4586,
        -1.4586, -0.4586])

In [7]:
iris = load_iris()

class Net(nn.Module):
    ## 個々で
    def __init__(self):
        ## nn.Moduleを継承するよってことで。
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 3)
    def forward(self, x):
        x = self.fc1(x)
        ## 最後にlog softmax(ココは普通にF.softmax()に変えても処理は回ります。)で出力
        return F.log_softmax(x, dim = 1)

In [21]:
model = Net()
train_X = torch.Tensor(iris.data)       ## torch.Tensor型: Pytorch版のnumpy形式
train_y = torch.LongTensor(iris.target)


In [40]:
optimizer = optim.SGD(model.parameters(), lr=0.02)
## エポック数(何回学習させるか)
epochs = 1000
## 後の可視化用に取っておく
loss_log, accuracy_log = [], []
w = []

data, target = Variable(train_X), Variable(train_y)
for epoch in range(epochs):
    ## pytorchでは勾配を蓄積する仕組みなので更新前に初期化しておきます
    optimizer.zero_grad()
    ## feed forward(つまり予測させます)
    output = model(data)
    ## 予実差からの誤差を決めます。nll_lossは、Negative Log Likelihood(負の対数尤度)ですね。
    loss = F.nll_loss(output, target)

    ## Back Propagation
    loss.backward()
    w.append(list(model.parameters())[0])
    # print(list(model.parameters())[0].grad)
    ## 各層内のパラメータの更新
    optimizer.step()


    ## 正解率の計算
    prediction = output.data.max(1)[1]
    accuracy = prediction.eq(target.data).sum().numpy() / len(iris.data)

    ## 記録を取っておきます
    loss_log.append(loss.item())
    accuracy_log.append(accuracy)

    if epoch % 100 == 0:
        print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(epoch, loss.data.item(), accuracy))

tensor([[-6.8543e-03, -1.4153e-02,  1.8965e-02,  9.0431e-03],
        [-7.1887e-03,  7.8239e-05,  3.8140e-03,  1.2924e-02],
        [ 1.4043e-02,  1.4075e-02, -2.2779e-02, -2.1967e-02]])
Train Step: 0	Loss: 0.198	Accuracy: 0.973
tensor([[-6.8519e-03, -1.4147e-02,  1.8958e-02,  9.0397e-03],
        [-7.1848e-03,  7.6629e-05,  3.8119e-03,  1.2919e-02],
        [ 1.4037e-02,  1.4071e-02, -2.2770e-02, -2.1959e-02]])
tensor([[-6.8494e-03, -1.4142e-02,  1.8951e-02,  9.0364e-03],
        [-7.1807e-03,  7.5124e-05,  3.8100e-03,  1.2914e-02],
        [ 1.4030e-02,  1.4067e-02, -2.2761e-02, -2.1950e-02]])
tensor([[-6.8469e-03, -1.4136e-02,  1.8943e-02,  9.0330e-03],
        [-7.1762e-03,  7.3738e-05,  3.8083e-03,  1.2909e-02],
        [ 1.4023e-02,  1.4063e-02, -2.2752e-02, -2.1942e-02]])
tensor([[-6.8444e-03, -1.4131e-02,  1.8936e-02,  9.0297e-03],
        [-7.1728e-03,  7.1920e-05,  3.8060e-03,  1.2904e-02],
        [ 1.4017e-02,  1.4059e-02, -2.2742e-02, -2.1934e-02]])
tensor([[-6.8419e-03, -

In [24]:
a = list(model.parameters())

In [27]:
a[0].grad

tensor([[-0.0108, -0.0233,  0.0311,  0.0147],
        [-0.0141,  0.0034,  0.0067,  0.0209],
        [ 0.0250,  0.0199, -0.0378, -0.0356]])

In [28]:
a[0]

Parameter containing:
tensor([[ 0.5973,  1.0892, -1.5722, -0.3232],
        [ 0.5507, -0.5255,  0.1543, -0.2600],
        [-0.4357, -1.2927,  1.5662,  0.9310]], requires_grad=True)

In [34]:

list(model.parameters())[0]

tensor(0.0399, grad_fn=<MeanBackward0>)

In [41]:
w

[Parameter containing:
 tensor([[ 0.8852,  1.6873, -2.3735, -0.7045],
         [ 0.8645, -0.5431, -0.0085, -0.8054],
         [-1.0374, -1.8731,  2.5302,  1.8577]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.8852,  1.6873, -2.3735, -0.7045],
         [ 0.8645, -0.5431, -0.0085, -0.8054],
         [-1.0374, -1.8731,  2.5302,  1.8577]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.8852,  1.6873, -2.3735, -0.7045],
         [ 0.8645, -0.5431, -0.0085, -0.8054],
         [-1.0374, -1.8731,  2.5302,  1.8577]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.8852,  1.6873, -2.3735, -0.7045],
         [ 0.8645, -0.5431, -0.0085, -0.8054],
         [-1.0374, -1.8731,  2.5302,  1.8577]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.8852,  1.6873, -2.3735, -0.7045],
         [ 0.8645, -0.5431, -0.0085, -0.8054],
         [-1.0374, -1.8731,  2.5302,  1.8577]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.8852,  1.6873, -2.3735, -0.7045],


In [44]:
torch.stack(w).shape

torch.Size([1000, 3, 4])